数学优化问题，也叫最优化问题，是指在一定约束条件下，求解一个目标函数的最大值（或最小值）问题。

给定一个目标函数（也叫代价函数）$f : A → R$，寻找一个变量（也叫参数）$x^∗∈ D$，使得对于所有$D$中的$x$，$f(x^∗) ≤ f(x)$（最小化）；或者$f(x^∗) ≥ f(x)$（最大化），其中$D$为变量$x$ 的约束集，也叫可行域；$D$中的变量被称为是可行解。

来看一个例子，比如目前已知5个点，第i个点记作$(x_i, y_i)$，并且这5个点是从某个2次函数上面选取出来的，现在需要求出这这个二次函数。

我们先假设这个二次函数为$y = ax^2+bx+c$，其中a，b，c都是这个函数的参数，我们要做的就是找到合适的参数。

这个参数要怎么找呢，先来看一下，如果我们把每个点的x值代入我们设出来的函数，则可以得到5个y，而当这5个由函数得到的y正好全部和5个点的y值相等时，说明我们找到的函数是正确的。

因此，我们可以设一个函数为：

### $$L(x, y, a, b, c) = \sum_i^5 (y_i - (ax_i^2+bx_i+c))^2$$

可以发现，只有当每个$y_i$都等于$ax_i^2+bx_i+c$时，该函数取最小值0。因此我们可以把问题变成一个优化问题：也就是固定x，y、寻找合适的a，b，c，使得函数L的取值最小。

如果大家对凸函数有所了解的话，可以发现我们定义的函数L，正好是一个凸函数，而凸函数如果存在极值点，一定是最小值点。因此这个问题我们可以直接对a, b, c求偏导并令其为0，最终可以得到a, b, c的值。

那么，这个问题怎么和我们的神经网络联系在一起呢？

上面所说的$y = ax^2+bx+c$，这个其实就等价于我们的模型，而模型呢，是由输入输出和可训练的参数构成，这里的a, b, c其实就可训练的参数，我们训练的目的就是去为模型寻找合适的参数，使模型在数据上能够有不错的表现。

我们这里的5个点，其实就是我们用于训练的数据，x表示训练输入，y表示训练标签，联想MNIST数据集，我们也有着对应的image和label。

而上面的L函数呢，我们称之为目标函数，或者损失函数。之前有提过这个函数是用来衡量我们模型预测与真实标签之间的差距，差距越小则目标函数的值越小。

我们常用到的损失函数主要有：
 - 1.我们之前使用过的mse，意思是均方误差，公式为$Loss = \frac{1}{2n}\sum_i^n (y_i - model(x_i))^2$，这个函数在分类和回归任务上都可以用，但一般更多用在回归任务上。
 - 2.交叉熵损失函数，我们在MNIST那里也用到过。交叉熵是从信息论中提取出的一个函数，大家只需要知道它的作用是衡量两个概率分布p ,q之间的差距。公式为$- \sum_i^n y_i\:log\:(model(x_i))$，因为它衡量的是概率分布，因此很适合于分类任务。
 
如果不引入模型来说，上面提到的两个函数都是凸函数。但是，如果我们引入了模型之后，这两个函数就不一定是凸函数了。

![](pic\2.jpg)

在上一节中我们提到了在神经元中引入非线性。非线性确实为我们的模型带来很大的提升，但是非线性层之间的多次复合变换，使得我们模型变得极为复杂，然后，我们又把损失函数套了上去，最终得到的东西，还能保持为凸的概率实在是太小了。

由于失去了凸这么一个优秀的性质，我们优化神经网络，没有办法能够1步到位，因为最终的目标函数上可能会存在许许多多的极小值点，就光是求出这些极小值点就已经极其困难了，我们还不能保证找到的极小值点一定是全局最小的。

不过还好有理论说，损失函数的大部分极小值点都和全局极小值足够接近。也就是说即使我们找到的极小值点不是全局极小值点，那大概率也和全局极小值点接近，即网络的表现还是可以让人满意的。

但是怎么去寻找一个极小值点呢？如果依靠令偏导数为0来求解，这将会得到一个无比恐怖的方程(就光我们实现的两层全连接MNIST识别，参数量就有11万多，大型神经网络的参数量有过万亿的)，想要解这种方程……即使能解出来，需要花费的时间就很长很长，肯定不是人能够忍受的。

因此我们可以换一个思路：我们不知道极小值点在哪里，但是我们如果能知道，我们从哪个方向走可以下降的话，那么一直走下去肯定可以走到一个极小值点(这里有误，也可能走到鞍点，但一般梯度下降效果都还不错)。恰好，梯度正好可以提供方向这个信息：梯度的本意是一个向量，表示某一函数在该点处的方向导数沿着该方向取得最大值，即函数在该点处沿着该方向增长最快。那么我们沿着梯度的相反方向，是不是就可以使得函数在该点处下降最快呢？

运用梯度信息来优化目标函数的方法称为梯度下降法(当然，不一定能找到全局极小值)

![](pic\1.gif)
![](pic\1.jpg)

如图，这个就是一个运用梯度下降来优化目标函数的例子，这里的目标函数比较简单，只是示意。

既然需要使用梯度下降法，我们就需要关注一下，哪些因素对其有影响。看上面两张图，我们可以发现，我们训练的过程中都会有一个起点，那么起点位置的好坏则会直接关系到我们最终落到的极小值的好坏。因此一开始处在一个比较好的初始点是对训练很重要的。那么，和起始点有关的是什么呢？

回忆一下我们模型的构成：输入输出和参数。输入输出是由训练数据决定，是固定的。因此决定模型起始点的因素就是模型的参数。在第三节，我们自己实现了一个全连接层，在那里，我们手动声明了两个参数，W和b，其中W是用一个服从高斯分布的初始化器初始化，而b是全1初始化。关于参数的初始化也有着很多的技巧，这里其实大家用随机的正态分布初始化都可以，均值一般为0，方差不能大。切记尽量不要把参数中(偏置可以)的每一个元素都初始化成相同的，因为这样会导致偏导数比较对称，训练相对困难。

说完了参数初始化的问题，再来看一下另一个问题：根据梯度下降法，我们只需要一直朝着梯度的反方向移动，但是每次我们移动的长度是多少比较合适呢？先引入一个概念，叫学习率(learning rate)，我们更新参数时(更新就是移动的过程)，参数的变化量 = rl * (-梯度)，如果学习率比较小，则训练会很慢，而如果学习率过大，则会导致我们在极小值周围乱跳：

![](pic\3.png)

我们也可以在之前提到的playground中尝试修改学习率，查看效果。

# 优化器

优化器，其实就是这么一个运用梯度下降原理来优化神经网络的东西。

最普通的优化器就是梯度下降优化器了，这种优化器是直接运用梯度下降法，因此没什么好说的。

而我们使用优化器时，一般有三种方法：
 - 1.全局梯度下降：每次投入所有的样本，计算梯度，优化模型。这样的训练是最稳妥的，因为，每次迭代都考虑到了所有的数据。但是如果数据量一旦大起来，首先占用内存大就是一个问题，其实每次这么多样本同时在模型中进行传播，计算耗时也是非常大的。
 - 2.随机梯度下降：梯度下降的一个反面就是随机梯度下降优化器了，这种方法每次只接受一个样本，也就是一个样本迭代更新一次。这样的话，内存占用就小了，并且速度快。但是相比梯度下降优化的稳定，随机梯度下降每次更新只考虑到一个样本，因此训练过程会更不稳定，耗时更长。
 - 3.批次梯度下降：将上面的两种优化器折中一下，就得到了我们的批次随机梯度下降。我们选定一个值，叫做batch_size，然后每次从训练数据种选取batch_size个值，进行训练。一般batch_size越大效果越好，但耗时也越多。当batch_size等于整个样本时，就是梯度下降。batch_size=1时就是SGD。
 
共性和个性。我们每个样本都存在着共性和个性，而我们学习，学习的不应该是个性，而是共性。选取批次，我们可以很好的平均个性，更好的去学习共性。

### 梯度下降的缺陷
梯度下降法因为只考虑了一阶导数，因此不能解决某些问题，一个比较著名的问题就是“鞍点”

![](pic\3.jpg)

看图，这个图像很想一个马鞍，而图像中间的点，就叫鞍点。这个点很特殊，我们在这里求梯度，得到的会是一个零向量。但是这个点只是其中某一个轴上的极小值点。对于这种问题，我们就不能只考虑一阶导数了。

目前来说keras支持的优化器种类很多，我们编译模型时，其实是默认使用了RMSprop这个优化器。如果没有什么其他的需求，一般情况使用RMSprop优化器或者Adam优化器就完全够了。


这里放两个图，大家看看不同优化器的表现：
![](pic\d75eefd9da37a334b47374a62a154336.gif)
![](pic\6f864a86cd1826ef53f2645ff305afd5.gif)